In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q tensorflow fair-esm transformers torch biopython numpy

In [ ]:
import os
import torch
import numpy as np
from tqdm import tqdm
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from transformers import T5EncoderModel, T5Tokenizer, BertModel, BertTokenizer
from Bio import SeqIO


#ESM-2

In [ ]:
!git clone https://github.com/facebookresearch/esm.git

In [ ]:
!python /content/esm/scripts/extract.py esm2_t33_650M_UR50D /content/drive/MyDrive/1149108/msCNN-ETC/datasets/tp_train_data.fasta /content/drive/MyDrive/1149108/msCNN-ETC/ --repr_layers 29 30 31 32 33 --include mean per_tok

In [ ]:
!python /content/esm/scripts/extract.py esm2_t33_650M_UR50D /content/drive/MyDrive/1149108/msCNN-ETC/datasets/tp_test_data.fasta /content/drive/MyDrive/1149108/msCNN-ETC/ --repr_layers 29 30 31 32 33 --include mean per_tok

In [ ]:
!python /content/esm/scripts/extract.py esm2_t33_650M_UR50D /content/drive/MyDrive/1149108/msCNN-ETC/datasets/et_test_data.fasta /content/drive/MyDrive/1149108/msCNN-ETC/ --repr_layers 29 30 31 32 33 --include mean per_tok

In [ ]:
!python /content/esm/scripts/extract.py esm2_t33_650M_UR50D /content/drive/MyDrive/1149108/msCNN-ETC/datasets/et_train_data.fasta /content/drive/MyDrive/1149108/msCNN-ETC/ --repr_layers 29 30 31 32 33 --include mean per_tok

In [ ]:
# configuration
maxlength = 1022
embedding_width = 1280
label_mapping = {'et': 1, 'tp': 0}
selected_layers = [29, 30, 31, 32, 33]

def process_folder(folder_path, label):
    embeddings = []
    labels = []

    files = [f for f in os.listdir(folder_path) if f.endswith('.pt')]

    for file in tqdm(files, desc=f"Processing {folder_path}"):
        data = torch.load(os.path.join(folder_path, file))

        layer_reps = []
        for layer in selected_layers:
            reps = data['representations'][layer]  # shape [L, embedding_width]
            layer_reps.append(reps)

        #  list to tensor 3D: [num_layers, L, embedding_width]
        stacked = torch.stack(layer_reps, dim=0)
        # mean [L, embedding_width]
        mean_reps = torch.mean(stacked, dim=0)

        # Padding zero
        padded = np.zeros((maxlength, embedding_width), dtype=np.float16)
        padded[:mean_reps.shape[0], :] = mean_reps.numpy()

        #  batch (1, maxlength, embedding_width)
        padded = padded[np.newaxis, ...]

        embeddings.append(padded)
        labels.append(label_mapping[label])

    return np.concatenate(embeddings, axis=0), np.array(labels)

X_train_et, y_train_et = process_folder('/content/drive/MyDrive/1149108/msCNN-ETC/et_train', 'et')
X_test_et, y_test_et = process_folder('/content/drive/MyDrive/1149108/msCNN-ETC/et_test', 'et')
X_train_tp, y_train_tp = process_folder('/content/drive/MyDrive/1149108/msCNN-ETC/tp_train', 'tp')
X_test_tp, y_test_tp = process_folder('/content/drive/MyDrive/1149108/msCNN-ETC/tp_test', 'tp')

X_train = np.concatenate([X_train_et, X_train_tp], axis=0)
y_train = np.concatenate([y_train_et, y_train_tp], axis=0)
X_test = np.concatenate([X_test_et, X_test_tp], axis=0)
y_test = np.concatenate([y_test_et, y_test_tp], axis=0)

np.save('/content/drive/MyDrive/1149108/msCNN-ETC/X_train.npy', X_train)
np.save('/content/drive/MyDrive/1149108/msCNN-ETC/y_train.npy', y_train)
np.save('/content/drive/MyDrive/1149108/msCNN-ETC/X_test.npy', X_test)
np.save('/content/drive/MyDrive/1149108/msCNN-ETC/y_test.npy', y_test)

print("Done!")
print(f"X_train shape (mean of layers): {X_train.shape}")
print(f"y_train shape: {y_train.shape}")

#ProtTrans

In [ ]:
LOG_DIR = '/content/drive/MyDrive/1149108/msCNN-ETC/ProtTrans'
os.makedirs(LOG_DIR, exist_ok=True)

In [ ]:
# Load ProtT5 model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = T5Tokenizer.from_pretrained("Rostlab/prot_t5_xl_uniref50", do_lower_case=False)
model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_uniref50")
model = model.to(device)
model = model.eval()

# Read fasta file with labels
def read_fasta_with_label(fasta_path):
    sequences = []
    labels = []
    with open(fasta_path, 'r') as f:
        seq = ''
        for line in f:
            if line.startswith('>'):
                if seq != '':
                    sequences.append(seq)
                    seq = ''
                label = int(line.strip().split('label=')[-1])
                labels.append(label)
            else:
                seq += line.strip()
        if seq != '':
            sequences.append(seq)
    return sequences, labels

# Embedding sequence
def embed_sequences(sequences, model, tokenizer, cut_length=1024, embedding_dim=1024):
    all_embeddings = []
    for idx, seq in enumerate(sequences):
        print(f"Embedding sequence {idx+1}/{len(sequences)}")

        # CCut sequence if longer than 1024 amino acids
        if len(seq) > cut_length:
            seq = seq[:cut_length]

        # ProtT5 expects spaces between each amino acid
        seq_spaced = ' '.join(list(seq))
        # Tokenize
        ids = tokenizer(seq_spaced, return_tensors="pt", padding="max_length", truncation=True, max_length=cut_length+2)
        input_ids = ids['input_ids'].to(device)
        attention_mask = ids['attention_mask'].to(device)

        # Encode
        with torch.no_grad():
            embedding = model(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state.squeeze(0)  # (seq_len, 1024)

        # Remove <s> and </s> token embeddings 
        embedding = embedding[1:-1]  #  (seq_len-2, 1024)

        # If still shorter than 1024, pad more
        if embedding.size(0) < cut_length:
            padding = torch.zeros((cut_length - embedding.size(0), embedding_dim), device=embedding.device)
            embedding = torch.cat([embedding, padding], dim=0)

        # Only take exactly 1024 vectors
        embedding = embedding[:cut_length, :]

        # Convert to numpy
        emb_np = embedding.cpu().numpy()  # (1024, 1024)

        # Add dimension (1, 1024, 1024)
        emb_np = np.expand_dims(emb_np, axis=0)

        all_embeddings.append(emb_np)

    all_embeddings = np.stack(all_embeddings)  # (samples, 1, 1024, 1024)
    return all_embeddings

# Save file
fasta_path = "/content/drive/MyDrive/1149108/msCNN-ETC/Merged-data-fasta/test_data.fasta"
sequences, labels = read_fasta_with_label(fasta_path)
embeddings = embed_sequences(sequences, model, tokenizer, cut_length=1024, embedding_dim=1024)

# Save embeddings and labels
np.save(f"{LOG_DIR}/X_test_prottrans.npy", embeddings)
np.save(f"{LOG_DIR}/y_test_prottrans.npy", np.array(labels))

Embedding sequence 1/1605
Embedding sequence 2/1605
Embedding sequence 3/1605
Embedding sequence 4/1605
Embedding sequence 5/1605
Embedding sequence 6/1605
Embedding sequence 7/1605
Embedding sequence 8/1605
Embedding sequence 9/1605
Embedding sequence 10/1605
Embedding sequence 11/1605
Embedding sequence 12/1605
Embedding sequence 13/1605
Embedding sequence 14/1605
Embedding sequence 15/1605
Embedding sequence 16/1605
Embedding sequence 17/1605
Embedding sequence 18/1605
Embedding sequence 19/1605
Embedding sequence 20/1605
Embedding sequence 21/1605
Embedding sequence 22/1605
Embedding sequence 23/1605
Embedding sequence 24/1605
Embedding sequence 25/1605
Embedding sequence 26/1605
Embedding sequence 27/1605
Embedding sequence 28/1605
Embedding sequence 29/1605
Embedding sequence 30/1605
Embedding sequence 31/1605
Embedding sequence 32/1605
Embedding sequence 33/1605
Embedding sequence 34/1605
Embedding sequence 35/1605
Embedding sequence 36/1605
Embedding sequence 37/1605
Embedding 

In [ ]:
# Load data
X_test = np.load(f'{LOG_DIR}/X_test_prottrans.npy')
y_test = np.load(f'{LOG_DIR}/y_test_prottrans.npy')


print("Data shape:")
print(f"X_test: {X_test.shape}")
print(f"y_test: {y_test.shape}")

#ProtBert

In [ ]:
LOG_DIR = '/content/drive/MyDrive/1149108/msCNN-ETC/ProtBert/'
os.makedirs(LOG_DIR, exist_ok=True)

In [ ]:
# Load ProtBert model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert_bfd", do_lower_case=False)
model = BertModel.from_pretrained("Rostlab/prot_bert_bfd")
model = model.to(device)
model = model.eval()

# Read fasta file with labels
def read_fasta_with_label(fasta_path):
    sequences = []
    labels = []
    with open(fasta_path, 'r') as f:
        seq = ''
        for line in f:
            if line.startswith('>'):
                if seq != '':
                    sequences.append(seq)
                    seq = ''
                label = int(line.strip().split('label=')[-1])
                labels.append(label)
            else:
                seq += line.strip()
        if seq != '':
            sequences.append(seq)
    return sequences, labels

# Embedding sequence
def embed_sequences(sequences, model, tokenizer, cut_length=1024, embedding_dim=1024):
    all_embeddings = []
    for idx, seq in enumerate(sequences):
        print(f"Embedding sequence {idx+1}/{len(sequences)}")

        # CCut sequence if longer than 1024 amino acids
        if len(seq) > cut_length:
            seq = seq[:cut_length]

        # ProtT5 expects spaces between each amino acid
        seq_spaced = ' '.join(list(seq))
        # Tokenize
        ids = tokenizer(seq_spaced, return_tensors="pt", padding="max_length", truncation=True, max_length=cut_length+2)
        input_ids = ids['input_ids'].to(device)
        attention_mask = ids['attention_mask'].to(device)

        # Encode
        with torch.no_grad():
            embedding = model(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state.squeeze(0)  # (seq_len, 1024)

        # Remove <s> and </s> token embeddings 
        embedding = embedding[1:-1]  #  (seq_len-2, 1024)

        # If still shorter than 1024, pad more
        if embedding.size(0) < cut_length:
            padding = torch.zeros((cut_length - embedding.size(0), embedding_dim), device=embedding.device)
            embedding = torch.cat([embedding, padding], dim=0)

        # Only take exactly 1024 vectors
        embedding = embedding[:cut_length, :]

        # Convert to numpy
        emb_np = embedding.cpu().numpy()  # (1024, 1024)

        # Add dimension (1, 1024, 1024)
        emb_np = np.expand_dims(emb_np, axis=0)

        all_embeddings.append(emb_np)

    all_embeddings = np.stack(all_embeddings)  # (samples, 1, 1024, 1024)
    return all_embeddings

# Save file
fasta_path = "/content/drive/MyDrive/1149108/msCNN-ETC/Merged-data-fasta/test_data.fasta"
sequences, labels = read_fasta_with_label(fasta_path)
embeddings = embed_sequences(sequences, model, tokenizer, cut_length=1024, embedding_dim=1024)

# Save embeddings and labels
np.save(f"{LOG_DIR}/X_test_protbert.npy", embeddings)
np.save(f"{LOG_DIR}/y_test_protbert.npy", np.array(labels))

Embedding sequence 1/1605
Embedding sequence 2/1605
Embedding sequence 3/1605
Embedding sequence 4/1605
Embedding sequence 5/1605
Embedding sequence 6/1605
Embedding sequence 7/1605
Embedding sequence 8/1605
Embedding sequence 9/1605
Embedding sequence 10/1605
Embedding sequence 11/1605
Embedding sequence 12/1605
Embedding sequence 13/1605
Embedding sequence 14/1605
Embedding sequence 15/1605
Embedding sequence 16/1605
Embedding sequence 17/1605
Embedding sequence 18/1605
Embedding sequence 19/1605
Embedding sequence 20/1605
Embedding sequence 21/1605
Embedding sequence 22/1605
Embedding sequence 23/1605
Embedding sequence 24/1605
Embedding sequence 25/1605
Embedding sequence 26/1605
Embedding sequence 27/1605
Embedding sequence 28/1605
Embedding sequence 29/1605
Embedding sequence 30/1605
Embedding sequence 31/1605
Embedding sequence 32/1605
Embedding sequence 33/1605
Embedding sequence 34/1605
Embedding sequence 35/1605
Embedding sequence 36/1605
Embedding sequence 37/1605
Embedding 

In [ ]:
# Load data
X_test = np.load(f'{LOG_DIR}/X_test_protbert.npy')
y_test = np.load(f'{LOG_DIR}/y_test_protbert.npy')


print("Data shape:")
print(f"X_test: {X_test.shape}")
print(f"y_test: {y_test.shape}")